### Inventory Forecasting Model

###### Acessing database to fetch datas

In [3]:
## Importing required libraries to fetch the data
import pymysql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Database connection details
host = "18.136.157.135"
port = 3306  
database = "project_service_data"
username = "dm_usdata_sql"
password = "37z<49REb&mKnl4AV!vJ"

In [5]:
# Establishing the connection
connection = pymysql.connect(
    host=host,
    port=port,
    user=username,
    password=password,
    database=database
)

In [6]:
# SQL Query to fetch data
sql_query = "SELECT * FROM service_data"

#### --Data Preprocessing

In [7]:
# Reading the Data
Inventory = pd.read_sql(sql_query, connection)

In [8]:
## Displaying the Top 5 
Inventory.head()

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
0,30-05-17,30-05-17,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL
1,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL
2,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH
3,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES
4,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL


In [9]:
## Displaying the bottom 5
Inventory.tail()

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
28477,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,SPROCKET RUBBER
28478,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,CLUTCH CABLE
28479,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,OIL FILTER
28480,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,DISC OIL
28481,06-01-19,06-01-19,pravXXXXXXXXXX,KA53EVXXXX,BAJAJ DISCOVER 125,12805,AIR FILTER


In [10]:
## Column info
Inventory.columns

Index(['invoice_date', 'job_card_date', 'business_partner_name', 'vehicle_no',
       'vehicle_model', 'current_km_reading', 'invoice_line_text'],
      dtype='object')

#### Column Overview

##### "invoice_date" & "job_card_date" (time-based data): These columns provide temporal information, which can be crucial for identifying seasonal patterns, trends over time, and lead times between job cards and invoicing.
##### "business_partner_name" & "vehicle_model" (categorical data): These columns represent entities or categories, such as the service provider or the type of vehicle, which could impact demand for different spare parts.
##### "vehicle_no" (unique identifier): This could help track specific vehicles' repair history but might not directly contribute to forecasting unless used to aggregate repair events.
##### "current_km_reading" (numerical data): This represents the vehicle's current mileage and can be useful for predicting when spare parts are likely to be needed (e.g., for vehicles with higher mileage).

In [13]:
## Checking for null values
Inventory.isnull().sum()

invoice_date             0
job_card_date            0
business_partner_name    0
vehicle_no               0
vehicle_model            0
current_km_reading       0
invoice_line_text        6
dtype: int64

In [14]:
Inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28482 entries, 0 to 28481
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   invoice_date           28482 non-null  object
 1   job_card_date          28482 non-null  object
 2   business_partner_name  28482 non-null  object
 3   vehicle_no             28482 non-null  object
 4   vehicle_model          28482 non-null  object
 5   current_km_reading     28482 non-null  object
 6   invoice_line_text      28476 non-null  object
dtypes: object(7)
memory usage: 1.5+ MB


###### With only 6 null values in the column "invoice_line_text".... Dropping the nulls

In [16]:
# Dropping the null
Inventory = Inventory.dropna()

In [17]:
Inventory.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28476 entries, 0 to 28481
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   invoice_date           28476 non-null  object
 1   job_card_date          28476 non-null  object
 2   business_partner_name  28476 non-null  object
 3   vehicle_no             28476 non-null  object
 4   vehicle_model          28476 non-null  object
 5   current_km_reading     28476 non-null  object
 6   invoice_line_text      28476 non-null  object
dtypes: object(7)
memory usage: 1.7+ MB


###### The null values and corresponding has been removed

##### Importing the required Libraries

In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import math

In [21]:
# Converting "invoice_date" and "job_card_date" into datetime format
Inventory['invoice_date'] = pd.to_datetime(Inventory['invoice_date'])
Inventory['job_card_date'] = pd.to_datetime(Inventory['job_card_date'])

In [22]:
# Feature Engineering: Extract time-based features
Inventory['invoice_day'] = Inventory['invoice_date'].dt.day
Inventory['invoice_month'] = Inventory['invoice_date'].dt.month
Inventory['invoice_weekday'] = Inventory['invoice_date'].dt.weekday

In [23]:
Inventory['invoice_year'] = Inventory['invoice_date'].dt.year

In [24]:
### Label encoder
label_encoder = LabelEncoder()

In [46]:
# Aggregate the data by invoice month, vehicle model, and invoice line text
# This will give us the monthly demand for each part
monthly_demand = Inventory.groupby(['invoice_month', 'invoice_line_text'])['invoice_line_text'].count().reset_index(name='monthly_demand')

In [50]:
monthly_demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2319 entries, 0 to 2318
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   invoice_month      2319 non-null   int32 
 1   invoice_line_text  2319 non-null   object
 2   monthly_demand     2319 non-null   int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 45.4+ KB


In [52]:
monthly_demand['invoice_line_text_encoded'] = label_encoder.fit_transform(monthly_demand['invoice_line_text'])

In [72]:
monthly_demand.shape

(2319, 4)

In [74]:
# Add month-based and other features
monthly_demand['month'] = monthly_demand['invoice_month']

In [76]:
monthly_demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2319 entries, 0 to 2318
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   invoice_month              2319 non-null   int32 
 1   invoice_line_text          2319 non-null   object
 2   monthly_demand             2319 non-null   int64 
 3   invoice_line_text_encoded  2319 non-null   int32 
 4   month                      2319 non-null   int32 
dtypes: int32(3), int64(1), object(1)
memory usage: 63.5+ KB


In [90]:
# Display the prepared data
monthly_demand.head()

,invoice_month,invoice_line_text,monthly_demand,invoice_line_text_encoded,month
0,1,,4,0,1
1,1,3M OIL,84,3,1
2,1,ACCELATOR CABLE,1,9,1
3,1,AIR FILTER,96,11,1
4,1,AIR FILTER CHECKUP,29,12,1


In [101]:
monthly_demand['Year'] = 2025

###### Created a  year column example = 2025

In [96]:
## Dropping the unencoded invoice_line_text
monthly_demand.drop('invoice_line_text', axis = 1, inplace = True)

In [108]:
monthly_demand.info() ## Remaining columns after dropping unnencoded invoice_line_text

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2319 entries, 0 to 2318
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   invoice_month              2319 non-null   int32
 1   monthly_demand             2319 non-null   int64
 2   invoice_line_text_encoded  2319 non-null   int32
 3   month                      2319 non-null   int32
 4   Year                       2319 non-null   int64
dtypes: int32(3), int64(2)
memory usage: 63.5 KB


#### Feature Engineering

##### Preparing features for Training

In [119]:
print(monthly_demand.columns)

Index(['invoice_month', 'monthly_demand', 'invoice_line_text_encoded', 'month',
       'Year'],
      dtype='object')


In [123]:
features = ['invoice_line_text_encoded', 'month', 'Year']
X = monthly_demand[features]
y = monthly_demand['monthly_demand']  # Here the dataframe name an dthe column(Target variable) are both same

In [125]:
# Split the data into training and testing sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [127]:
# Display the shape of the datasets
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (1855, 3)
Testing data shape: (464, 3)


#### Model Building

In [130]:
# Train RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [132]:
# Evaluate the RandomForest model
rf_train_score = rf_model.score(X_train, y_train)
rf_test_score = rf_model.score(X_test, y_test)

In [134]:
print(f"Random Forest Model Train Score: {rf_train_score}")
print(f"Random Forest Model Test Score: {rf_test_score}")

Random Forest Model Train Score: 0.9865111489152264
Random Forest Model Test Score: 0.8848819068511087


##### Train XGBoost Model

In [137]:
# Train XGBoost Regressor
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [139]:
# Evaluate the XGBoost model
xgb_train_score = xgb_model.score(X_train, y_train)
xgb_test_score = xgb_model.score(X_test, y_test)

In [141]:
print(f"XGBoost Model Train Score: {xgb_train_score}")
print(f"XGBoost Model Test Score: {xgb_test_score}")

XGBoost Model Train Score: 0.859746298661354
XGBoost Model Test Score: 0.49509141641644494


##### Evaluating models with RMSE (Root Mean Squared Error)

In [144]:
# Predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)

In [150]:
# Calculate RMSE for both models
rmse_rf = math.sqrt(mean_squared_error(y_test, y_pred_rf))
rmse_xgb = math.sqrt(mean_squared_error(y_test, y_pred_xgb))

In [152]:
print(f"Random Forest RMSE: {rmse_rf}")
print(f"XGBoost RMSE: {rmse_xgb}")

Random Forest RMSE: 11.516022954921368
XGBoost RMSE: 24.117767559271122


##### Based on RMSE, the Random Forest model is the better one, as it has a lower RMSE, indicating it makes more accurate predictions compared to XGBoost.

### -----------------------------------------------------------------------------------------------------------------------------

##### Forecasting Future Demand Example

In [166]:
# Example: Forecast demand for the next month
new_data = pd.DataFrame({
    'invoice_line_text_encoded': [label_encoder.transform(['ENGINE OIL'])[0]],  # Example: "ENGINE OIL"
    'month': [2],  # Example: February
    'Year': [2025]  # Example: Forecasting for the year 2025
})

In [168]:
# Predict demand using Random Forest
forecast_rf = rf_model.predict(new_data)
# Predict demand using XGBoost
forecast_xgb = xgb_model.predict(new_data)

In [170]:
print(f"Forecast using Random Forest: {forecast_rf[0]}")
print(f"Forecast using XGBoost: {forecast_xgb[0]}")

Forecast using Random Forest: 278.52
Forecast using XGBoost: 259.15557861328125


#### -----------------------------------------------------------------------------------------------------------------------

##### - The models are providing slightly different predictions. Random Forest is forecasting higher demand, while XGBoost is forecasting lower demand.
##### - Based on RMSE, the Random Forest model is the better one, as it has a lower RMSE, indicating it makes more accurate predictions compared to XGBoost.